# serving a test endpoint with the `IPython` shell

its very possible to prototype web applications in the notebook.
they actually provide a really flexible surface for development.
interactively we can:

1. start a web server
2. interactively modify the server endpoints
3. run as a standalone server
4. make requests to the server

```mermaid
flowchart LR
    fastapi.FastAPI ---> uvicorn.Config --> uvicorn.Server
    httpx.ASGITransport -- transport --> httpx.AsyncClient
    fastapi.FastAPI -- app --> httpx.ASGITransport
```

with the feature installed it becomes a lot easier to develop templates
and front end features in both parts and as the whole.

In [1]:
import hashlib, fastapi.templating, uvicorn
from schema_frame import tag
shell.display_formatter.formatters["text/html"].for_type("bs4.element.Tag", str);

__import__("nest_asyncio").apply()

In [2]:
@dataclass
class WebApp:
    app: fastapi.FastAPI = None
    server: uvicorn.Server = None
    favicon: Path = None
    task: asyncio.Task = None
    stack = None

    def __post_init__(self):
        self.app = FastAPI()
        self.server = uvicorn.server.Server(uvicorn.Config(self.app))
        self.app.get("/favicon.ico")(self.get_favicon)

    async def __aenter__(self):
        self.stack = contextlib.AsyncExitStack()
        await self.stack.__aenter__()
        return await self.stack.enter_async_context(
            httpx.AsyncClient(transport=httpx.ASGITransport(self.app))
        )
    def __aexit__(self, *args):
        return self.stack.__aexit__(*args)

    def __enter__(self):
        self.stack = contextlib.ExitStack()
        self.stack.__enter__()
        return self.stack.enter_context(
            httpx.Client(transport=httpx.ASGITransport(self.app))
        )
        
    def __exit__(self, *args):
        return self.stack.__exit__(*args)
        
    async def stop(self):
        try:
            await self.server.shutdown()
        except AttributeError: pass

    def start(self):
        self.task = asyncio.ensure_future(self.server.serve())

    @property
    def url(self):
        return F"http://{self.server.config.host}:{self.server.config.port}"

    def _ipython_display_(self):
        print(self.url, repr(self))
    
    def iframe(self, path="", open=False, width="100%", height=600):
        url = self.url + path or ""
        attrs = {}
        if open:
            attrs["open"] = "true"
        return tag.details(
            tag.summary(url),
            tag.iframe(src=self.url + path or "", width=width, height=height, loading="lazy"), **attrs
        )

    def get_favicon(self):
        url = urllib.parse.urlparse(self.favicon)
        if url.scheme and url.scheme != "file":
            return fastapi.responses.RedirectResponse(self.favicon)
        return fastapi.responses.FileResponse(self.favicon)
        
    
        

`fastapi` appends routes, and retrieves the first matching routes. as a result, new appended routes are not hit. 
this custom `fastapi.FastAPI` provides conveniences for interactive development. there is little reason for it to be 
used outside of interactive development.

In [3]:
class FastAPI(fastapi.FastAPI):
    def verb(self, path, *args, _method="get", **kwargs):
        object = getattr(super(), _method)(path, *args, **kwargs)
        pops = []
        for i, route in enumerate(self.routes):
            if route.path == path:
                if _method.upper() in route.methods:
                    pops.append(i)    
        for i in reversed(pops):
            self.routes.pop(i)
        return object

    def template_response(self, *args, **kwargs):
        return self.template_environment().TemplateResponse(*args, **kwargs)
    
    def template_environment(self, *args, **kwargs):
        from IPython import get_ipython
        shell = get_ipython()
        return fastapi.templating.Jinja2Templates("", env=shell.env)
        
    responses = fastapi.responses
    for v in "get head post put delete options trace patch".split():
        locals().update({v: functools.partialmethod(
            functools.wraps(getattr(fastapi.FastAPI, v))(verb), _method=v
        )})

In [4]:
if "self" in locals(): 
    await self.stop()
app = (self := WebApp(favicon="https://a11yhood.org/images/ahood-200.jpg")).app
self.start()

INFO:     Started server process [20762]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


In [5]:
@compose_left(app.get(""), app.get("/"))
def hello(request: fastapi.Request):
    return app.responses.HTMLResponse(shell.env.get_template("hello.md").render(df=df, **vars(builtins)))

In [6]:
df = pandas.read_json("../../../research/at.json.gz")

df = df.assign(id=(h := df.index.to_series().apply(
        compose_left(str.encode, hashlib.sha256, operator.methodcaller("hexdigest"), take(10), "".join)
))).reset_index().set_index("id")

In [7]:
%%

    do(Path("hello.md").write_text)\
# hello world

we're working with a dataframe that has {{len(df)}} rows.

{{df.sample(2)._repr_html_()}}

In [8]:
display(self.iframe(""))

<details><summary>http://127.0.0.1:8000</summary><iframe height="600" loading="lazy" src="http://127.0.0.1:8000" width="100%"></iframe></details>

In [9]:
%%

    print\
```pug
- var ids = df.index[:2]
form
ol
    for id in ids
        - var item = df.loc[id]
        li.item
            header
            section
                h2: a(href=item.name) !{item["name"]}
                ol.row.tags
                    each tag in item.tags
                        li: a !{tag}
                section.description !{item["description"]}
            footer
style.
    .row {display: flex;}
    ol.tags li {list-style: none;}
```

- var ids = df.index[:2]
form
ol
    for id in ids
        - var item = df.loc[id]
        li.item
            header
            section
                h2: a(href=item.name) !{item["name"]}
                ol.row.tags
                    each tag in item.tags
                        li: a !{tag}
                section.description !{item["description"]}
            footer
style.
    .row {display: flex;}
    ol.tags li {list-style: none;}



## accessing data from a dataframe

In [10]:
# /item?id=9e3563d58e&id=5d95dc1297
@app.get("/item", response_class=fastapi.responses.HTMLResponse)
@app.get("/item/index.html", response_class=fastapi.responses.HTMLResponse)
def items(id: list[str]=None):
    if id:
        pass
    return str(
        tag.ol(*(tag.li(tag.a(x, href=F"/item/{x}")) for x in df.index))
    )

@app.get("/item/{id}")
def items(request: fastapi.Request, id: str):
    global req
    req = request
    return df.loc[[id]].reset_index().iloc[0].to_dict()

In [11]:
display(self.iframe("/item"), self.iframe("/item/" + df.index[0]))

<details><summary>http://127.0.0.1:8000/item</summary><iframe height="600" loading="lazy" src="http://127.0.0.1:8000/item" width="100%"></iframe></details>

<details><summary>http://127.0.0.1:8000/item/9e3563d58e</summary><iframe height="600" loading="lazy" src="http://127.0.0.1:8000/item/9e3563d58e" width="100%"></iframe></details>

## making requests to the server

In [12]:
import httpx
async with self as client:
    responses = (self.url + "/item/" + df.index.to_series().head()).apply(client.get).gather()

responses.apply(httpx.Response.json).series()

,id,index,description,stargazerCount,forkCount,license,tags,name
id,,,,,,,,
9e3563d58e,9e3563d58e,https://github.com/ai-collection/ai-collection,The Generative AI Landscape - A Collection of ...,7916.0,780.0,MIT License,"[artificial intelligence, collections, ai, ass...",ai-collection
5d95dc1297,5d95dc1297,https://github.com/OptiKey/OptiKey,OptiKey - Full computer control and speech wit...,4314.0,507.0,GNU General Public License v3.0,"[eye tracking, eyetracking, eye tracker, eyes,...",OptiKey
078e30ecc2,078e30ecc2,https://github.com/brunopulis/awesome-a11y,A curate list about A11Y,1843.0,145.0,Creative Commons Zero v1.0 Universal,"[accessibility, wai aria, wcag, a11y, awesome ...",awesome-a11y
a974b26bfe,a974b26bfe,https://github.com/Stypox/dicio-android,Dicio assistant app for Android,886.0,81.0,GNU General Public License v3.0,"[assistant, assistive technology, personal ass...",dicio-android
198987f80d,198987f80d,https://github.com/cboard-org/cboard,Augmentative and Alternative Communication (AA...,667.0,180.0,GNU General Public License v3.0,"[aac, autism, cerebral palsy, progressive web ...",cboard
